# Day 16

After a good try on the bit wise operation I reslied that we need at least a clock plus or an inverted signal.  
So now let do it with string manipulation


In [235]:
def get_data(filename):
    with open(filename) as f:
        return int(f.readline(), 16)


def get_bins(value):
    line = bin(value).replace("0b", "")
    if len(line) % 4 != 0:
        line = "0" * (4 - (len(line) % 4)) + line
    return line


def bin_to_int(string: str):
    if string.isnumeric():
        return int(string, 2)
    return 0


data = get_data("input")
# data


In [371]:
def sub_string(string: str, at: int):
    return string[0:at], string[at:]


phase = "init"


def literal(string: str):
    def literal_value(packets):
        return int("".join([f"{bin_to_int(p):04b}" for p in packets]), 2)

    hasnext = "1"
    data_stack = []
    while hasnext == "1":
        hasnext, string = sub_string(string, 1)
        # print(hasnext)
        d, string = sub_string(string, 4)
        data_stack.append(d)

    return literal_value(data_stack), string, 5 * len(data_stack)


def get_versions(hexval: int):
    data = get_bins(hexval)
    versions = []
    while len(data) > 0 and "1" in data:
        ver, data = sub_string(data, 3)
        typ, data = sub_string(data, 3)
        versions.append(bin_to_int(ver))
        if typ == "100":  # 4 == 0b0100
            _, data, _, = literal(data)
            # print('literal', a)
        else:
            # print('operation', bin_to_int(typ))
            inst, data = sub_string(data, 1)
            if inst == "1":
                _, data = sub_string(data, 11)
            else:
                _, data = sub_string(data, 15)
    return versions


# print(versions)
# print(sum(versions))

test_data = [
    (0x38006F45291200, 9),
    (0xEE00D40C823060, 14),
    (0x8A004A801A8002F478, 16),
    (0x620080001611562C8802118E34, 12),
    (0xC0015000016115A2E0802F182340, 23),
    (0xA0016C880162017C3686B18A3D4780, 31),
]

for data, expected in test_data:
    res = get_versions(data)
    print(sum(res) == expected, expected, res)


True 9 [1, 6, 2]
True 14 [7, 2, 4, 1]
True 16 [4, 1, 5, 6]
True 12 [3, 0, 0, 5, 1, 0, 3]
True 23 [6, 0, 0, 6, 4, 7, 0]
True 31 [5, 1, 3, 7, 6, 5, 2, 2]


In [267]:
data = get_data("input")
res = get_versions(data)
print(sum(res), res)


936 [3, 7, 7, 3, 5, 3, 1, 4, 6, 5, 4, 0, 0, 7, 5, 7, 1, 5, 6, 1, 7, 0, 3, 2, 5, 4, 1, 2, 0, 6, 4, 7, 0, 7, 0, 2, 3, 4, 2, 1, 4, 1, 5, 3, 7, 0, 2, 7, 2, 2, 2, 3, 0, 7, 4, 0, 3, 5, 4, 6, 0, 0, 3, 4, 3, 5, 7, 2, 1, 2, 2, 5, 4, 2, 6, 1, 0, 0, 7, 2, 0, 5, 3, 5, 3, 2, 1, 1, 4, 4, 6, 5, 3, 3, 1, 2, 1, 3, 7, 0, 1, 1, 7, 0, 2, 2, 5, 5, 1, 4, 3, 5, 1, 6, 4, 7, 5, 0, 4, 7, 1, 1, 6, 3, 7, 0, 3, 0, 0, 2, 6, 7, 0, 3, 1, 3, 0, 2, 3, 1, 6, 5, 6, 4, 3, 5, 4, 4, 7, 6, 7, 3, 4, 1, 3, 2, 3, 2, 5, 0, 0, 1, 7, 3, 5, 4, 2, 6, 2, 7, 1, 2, 3, 2, 6, 7, 0, 6, 7, 3, 1, 5, 5, 7, 2, 6, 3, 6, 7, 6, 2, 4, 6, 5, 7, 4, 0, 7, 4, 7, 0, 6, 5, 7, 1, 3, 5, 2, 1, 7, 5, 4, 0, 6, 5, 2, 1, 7, 5, 7, 3, 5, 4, 6, 1, 0, 2, 7, 3, 6, 1, 0, 0, 5, 2, 3, 2, 7, 2, 7, 0, 1, 0, 3, 5, 0, 5, 6, 5, 6, 3, 3, 5, 0, 6, 5, 5, 5, 7, 2, 3, 4, 6, 2, 3, 2, 4, 6]


### Part 2 
After my failure with bit wise solution I just kinda give up on the solution.   
after a bit of googling I found a blog, where the blog used [this solution](https://github.com/SiddhantAttavar/Competitive-Programming/blob/main/Other%20Contests/Advent%20Of%20Code%202021/Day16/Day16Q2.py).



In [412]:
with open("input") as f:
    data = f.read().splitlines()

from functools import reduce

funcDict = {
    0: sum,
    1: lambda a: reduce(lambda x, y: x * y, a),
    2: min,
    3: max,
    5: lambda a: int(a[0] > a[1]),
    6: lambda a: int(a[0] < a[1]),
    7: lambda a: int(a[0] == a[1]),
}

def evaluate(unit):
    global packets
    global graph

    if packets[unit][1] == 4:
        return packets[unit][-2]

    res = []
    for v in graph[unit]:
        res.append(evaluate(v))
    return funcDict[packets[unit][1]](res)


def str_length_correction(string: str):
    for i in data[0]:
        if i != "0":
            break
        string = "0" * 4 + string
    n = len(string)
    if n % 4 != 0:
        string = "0" * (4 - n % 4) + string
    return string


def get_packets(string_data: str):
    current_count = 0
    packets = []
    while current_count < len(string_data) and "1" in string_data[current_count:]:
        ver = int(string_data[current_count : current_count + 3], 2)
        current_count += 3
        typ = int(string_data[current_count : current_count + 3], 2)
        current_count += 3

        if typ == 4:
            num = ""
            while string_data[current_count] == "1":
                num += string_data[current_count + 1 : current_count + 5]
                current_count += 5
            num += string_data[current_count + 1 : current_count + 5]
            current_count += 5
            num = int(num, 2)

            packets.append((ver, typ, 0, num, current_count))
        else:
            l_flag = int(string_data[current_count], 2)
            current_count += 1
            if l_flag == 0:
                num = int(string_data[current_count : current_count + 15], 2)
                current_count += 15
            else:
                num = int(string_data[current_count : current_count + 11], 2)
                current_count += 11

            packets.append([ver, typ, l_flag, num, current_count])
    return packets


s_bin = str_length_correction(bin(int(data[0], 16))[2:])
packets = get_packets(s_bin)
stack = []
graph = [[] for _ in range(len(packets))]

for i, (u_ver, u_typ, u_l_flag, u_num, u_curr_ix) in enumerate(packets):
    if len(stack) > 0:
        p = stack[-1]

        graph[p].append(i)
        packets[p][3] -= 1
        if packets[p][3] == 0:
            stack.pop()

    while len(stack) > 0:
        p = stack[-1]
        if packets[p][2] == 0 and packets[p][3] <= u_curr_ix - packets[p][-1]:
            stack.pop()
        else:
            break

    if u_typ != 4:
        stack.append(i)


print(f"Awnser: {evaluate(0)}")

Awnser: 6802496672062
